In [1]:
# Importación de Librerías Principales para el Análisis
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sb

In [2]:
# Lectura de Dataset 

# Arreglo con los nombres de las columnas que tienen distintos tipos
columnas_prob = ['approxdate', 'resolution', 'attacktype2_txt', 'attacktype3_txt', 
                 'gsubname2', 'gname3', 'gsubname3', 'claimmode2_txt',
                 'claimmode3_txt', 'weaptype3_txt', 'weapsubtype3_txt', 
                 'weaptype4_txt', 'weapsubtype4_txt', 'divert', 'kidhijcountry', 
                 'ransomnote']
tipos = dict.fromkeys(columnas_prob, str)

# Creación de dataframe inicial en base al csv 
data_inicial = pd.read_csv(r"./terrorism.csv", encoding='latin', dtype=tipos)

In [3]:
# Dimensiones del dataset 
print(data_inicial.shape)

(181691, 135)


In [4]:
# Agarrar solo los registros del 2012 en adelante 
# Esto se hace porque START empezó a recopilar datos formalmente desde esta fecha 
# Incluso se puede verificar que el total de registros de 2012 a 2018 son el 40%, proporcionalmente no se puede comparar 
# con los registros anterirores. 
data_general = data_inicial[data_inicial['iyear'] >= 2012]
data_general.shape

(76913, 135)

In [5]:
# Verificar cuántos nulos hay por columna para ver si hay alguna que definitivamente hay que eliminar 

# Eliminación de Columnas que son completamente nulas
# Además verifiqué cuales variables que tienen más del 60% de los datos nulos son importantes y cuales no. 
# importantes: alternative, alternative_txt, claimmode, claimmodetxt, 
todavia_no = ['alternative', 'alternative_txt']
for i in data_general:
    cantidad = data_general[i].isna().sum()
    proporcion = np.round(cantidad / data_general.shape[0],2)
    if proporcion >= 0.6 and i not in todavia_no:
        data_general.drop(i, 1, inplace=True)
print(data_general.shape)

d:\cs7\machine learning models\mlenv\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(76913, 62)


In [6]:
# Eliminar las variables que no se pueden utilizar, ej. descripciones 
data_preliminar = data_general.copy()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data_preliminar.head(50)

eliminar = ['location', 'summary', 'propcomment','scite1',
            'scite2', 'dbsource','corp1', 'target1', 'doubtterr']
data_preliminar.drop(eliminar, 1, inplace=True)

# Tratar el tema de alternative, como esta nulo por los que son 1s en la de doubtrr, ponerle 0 a los nas 
data_preliminar.loc[data_preliminar['alternative'].isna()] = 0

In [7]:
data_preliminar.isna().sum()

eventid                0
iyear                  0
imonth                 0
iday                   0
extended               0
country                0
country_txt            0
region                 0
region_txt             0
provstate              1
city                   0
latitude             111
longitude            111
specificity            0
vicinity               0
crit1                  0
crit2                  0
crit3                  0
alternative            0
alternative_txt        0
multiple               0
success                0
suicide                0
attacktype1            0
attacktype1_txt        0
targtype1              0
targtype1_txt          0
targsubtype1          54
targsubtype1_txt      54
natlty1               14
natlty1_txt           14
gname                  0
guncertain1            0
individual             0
nperps                 0
nperpcap               0
claimed                0
weaptype1              0
weaptype1_txt          0
weapsubtype1        1906


In [8]:
# imputacion de promedio sobre las variables numericas 
numeric = ['nkill', 'nkillus', 'nkillter', 'nwound',
           'nwoundte', 'nwoundus']
values = {i: np.round(data_preliminar[i].mean()) for i in numeric}
data_preliminar.fillna(value=values, inplace=True)

# imputacion de categoricas con una categoria adicional 
valores = {
    'targsubtype1': 0,
    'targsubtype1_txt':'unknown',
    'natlty1': 0, 
    'natlty1_txt':'unknown',
    'weapsubtype1':0,
    'weapsubtype1_txt':'unknown'
}
data_preliminar.fillna(value=valores, inplace=True)
data_preliminar.dropna(inplace=True)
print("Datos nulos: ", data_preliminar.isna().sum().sum())
data_preliminar.shape
# Finalmente no hay mas datos nulos

Datos nulos:  0


(76801, 53)

In [9]:
# Verificar la cantidad de valores unicos por columna 
data = data_preliminar.copy()

print("Datos únicos por columna")
for i in data:
    print(i, " -> ", len(data[i].unique()))

Datos únicos por columna
eventid  ->  14233
iyear  ->  7
imonth  ->  13
iday  ->  32
extended  ->  2
country  ->  95
country_txt  ->  95
region  ->  13
region_txt  ->  13
provstate  ->  635
city  ->  5461
latitude  ->  7598
longitude  ->  7520
specificity  ->  5
vicinity  ->  2
crit1  ->  2
crit2  ->  2
crit3  ->  2
alternative  ->  6
alternative_txt  ->  6
multiple  ->  2
success  ->  2
suicide  ->  2
attacktype1  ->  10
attacktype1_txt  ->  10
targtype1  ->  19
targtype1_txt  ->  19
targsubtype1  ->  93
targsubtype1_txt  ->  94
natlty1  ->  103
natlty1_txt  ->  104
gname  ->  347
guncertain1  ->  2
individual  ->  2
nperps  ->  56
nperpcap  ->  19
claimed  ->  2
weaptype1  ->  9
weaptype1_txt  ->  9
weapsubtype1  ->  30
weapsubtype1_txt  ->  31
nkill  ->  99
nkillus  ->  8
nkillter  ->  70
nwound  ->  77
nwoundus  ->  15
nwoundte  ->  35
property  ->  3
ishostkid  ->  3
INT_LOG  ->  3
INT_IDEO  ->  3
INT_MISC  ->  2
INT_ANY  ->  3
